In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Data import

In [ ]:
train = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/train.csv.zip")
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/test.csv.zip")
test.head()

In [ ]:
bids = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/bids.csv.zip")

In [ ]:
bids.head(50)

In [ ]:
bids.sort_values(by='bidder_id' ,ascending = True)

In [ ]:
bids["bidder_id"].value_counts()

In [ ]:
alldata = pd.concat([train,test])
alldata.shape

In [ ]:
alldata["payment_account"] = alldata["payment_account"].apply(lambda x:x[:5])

In [ ]:
alldata.head()

In [ ]:
alldata["payment_account"].value_counts()

In [ ]:
alldata["address"].sort_values().head(30)

In [ ]:
alldata["address"] = alldata["address"].apply(lambda x:x[:5])

In [ ]:
alldata.head()

In [ ]:
alldata["address"].value_counts()

In [ ]:
x = bids.groupby("bidder_id")["bid_id"].count().reset_index()  # bidder_id를 기준으로 bid_id가 몇번 나왔는지 카운트
x

In [ ]:
alldata = alldata.merge(x,on = "bidder_id",how = "left")

In [ ]:
alldata

In [ ]:
alldata["bid_id"] = alldata["bid_id"].fillna(0)
alldata

In [ ]:
alldata.isna().sum()

In [ ]:
alldata3 = alldata[alldata["bid_id"]<20000]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20,12))
sns.boxplot(alldata3["outcome"],alldata3["bid_id"])

In [ ]:
plt.figure(figsize = (20,12))
sns.boxplot(alldata3["outcome"],alldata3["device"])

In [ ]:
n_unique = bids.groupby("bidder_id")["auction","device","country"].nunique().reset_index()

In [ ]:
alldata = alldata.merge(n_unique,on = "bidder_id",how = "left")
alldata.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata["payment_account"] = le.fit_transform(alldata["payment_account"])

In [ ]:
alldata.head()

In [ ]:
alldata["address"] = le.fit_transform(alldata["address"])

In [ ]:
alldata.head()

In [ ]:
from datetime import datetime
bids["datetime"] = bids["time"].apply(lambda x: datetime.fromtimestamp(x/7000000))

In [ ]:
bids["year"] = bids["datetime"].dt.year
bids["month"] = bids["datetime"].dt.month
bids["day"] = bids["datetime"].dt.day

In [ ]:
year_mean = bids.groupby("bidder_id")["year"].mean().reset_index()
month_mean = bids.groupby("bidder_id")["month"].mean().reset_index()
day = bids.groupby("bidder_id")["day"].mean().reset_index()

In [ ]:
alldata = alldata.merge(year_mean, on = "bidder_id", how = "left")
alldata = alldata.merge(month_mean, on = "bidder_id", how = "left")
alldata = alldata.merge(day, on = "bidder_id", how = "left")
alldata.head()

In [ ]:
time_diff = bids.groupby("bidder_id")["time"].apply(lambda x : x.max() - x.min()).reset_index()
alldata = alldata.merge(time_diff, on = "bidder_id",how = "left")
alldata.head()

In [ ]:
alldata["time"] = alldata["time"]/alldata["bid_id"]

In [ ]:
alldata2 = alldata.drop(["bidder_id","outcome"],axis = 1)

In [ ]:
alldata2.head()

In [ ]:
alldata2 = alldata2.fillna(-1)


In [ ]:
train2 = alldata2[:len(train)]
train2.shape

In [ ]:
test2 = alldata2[len(train):]
test2.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 170, max_depth = 7)      # n_estimators : 트리의 갯수 지정
rf.fit(train2,train["outcome"])                       # train2 데이터에서 count열을 기준으로 모델 학습  
result = rf.predict_proba(test2)          

In [ ]:
result = result[:,1]

In [ ]:
train["outcome"].value_counts()

In [ ]:
sub = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/sampleSubmission.csv")
sub.head()

In [ ]:
sub["prediction"] = result
sub.to_csv("sub.csv",index = False)